In [2]:
# Install pymongo package
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
install("pymongo")

In [4]:
import pymongo
from pymongo import MongoClient

# Connect to MongoDB
mongoclient = pymongo.MongoClient("mongodb://localhost:7000/")

#Check what databases exist - the output is a list of database names
print(mongoclient.list_database_names())


['admin', 'config', 'local']


In [5]:
#Defining a user function to check if database exists - In MongoDB, a database is not created until it gets content. 
def check_DatabaseExists(argDBName):
    local_dblist = mongoclient.list_database_names()
    if argDBName in local_dblist:
        print("The database ", argDBName, " exists.")
    else:
        print("The database ", argDBName, " does not exist.")

#Defining a user function to check if a collection exists - In MongoDB, a collection is not created until it gets content. 
def check_CollectionExists(argDBName, argCollName, local_mydb):
    local_collist = local_mydb.list_collection_names()
    if argCollName in local_collist:
        print("The collection ",  argCollName, "exists in database ", argDBName)
    else:
        print("The collection ", argCollName, " does not exist in database ", argDBName)

In [6]:
#Create a new database       
mydb = mongoclient["GroupB_Supermarket"]
print(type(mydb))

#Check if database exists by calling function check_DatabaseExists with name of database as the arguement
check_DatabaseExists("GroupB_Supermarket")
print()

<class 'pymongo.database.Database'>
The database  GroupB_Supermarket  does not exist.



In [7]:
#Return a list of all collections in your database:
print(mydb.list_collection_names())

[]


In [8]:
#Create a new collection called "productDetails"
mycol = mydb["ProductDetails"]
print(type(mycol))

#Check if collection exists by calling function check_CollectionExists with the following arguements (parameters):
#Name of database as the first arguement 
#Name of collection as the second arguement
#mydb as the third arguement
#In MongoDB, a collection is not created until it gets content. 
check_CollectionExists("GroupB_Supermarket", "ProductDetails", mydb)
print()

<class 'pymongo.collection.Collection'>
The collection  ProductDetails  does not exist in database  GroupB_Supermarket



In [10]:
import csv

with open("S:\grocery_random_data_mongo.csv", "r") as csvfile:
    reader = csv.DictReader(csvfile)
    data = list(reader)  # Read all rows into a list of dictionaries

In [11]:
for document in data:
    mycol.insert_one(document)

In [12]:
#Show documents in collection "ProductDetails"
'''myresult = mycol.findone()

#print the result:
for x in myresult:
    print(x)'''
    
#The find_one() method returns the first occurrence in the selection.
x = mycol.find_one()
print(x)

{'_id': ObjectId('65fd836a7b2f6691a83bbf8c'), 'product_id': 'M10000', 'product_name': 'Pasta', 'price_in_pounds': '9.65', 'category_id': '105', 'category_name': 'Cereal & Bread', 'supermarket_id': '4', 'supermarket_name': 'Morrisons', 'image_url': 'https://example.com/products/M10000.jpg', 'description': 'Pasta from local farms', 'dietary_restrictions': '', 'nutritional_information': "{'calories': 434, 'fat': 1.6545525114670623, 'protein': 14.407659815352256, 'carbohydrates': 24.20258901687363}", 'Offers': 'No offers stay tuned', 'minimum_order_price': '40', 'customer_tags': 'family-friendly', 'best_before_date': '04/04/2024'}


In [13]:
#order price for supermarket 35 pounds and below 35 pounds
myquery = {"$or":[ {"supermarket_name": { "$regex": "^A" }}, {"minimum_order_price": { "$lte": '35' }}]}

    
pipeline = [{"$match": myquery},{"$group": {"_id": "$supermarket_name","minimum_order_price": {"$first": "$minimum_order_price"}}}]

distinct_records = mycol.aggregate(pipeline)

for record in distinct_records:
    print(record)

{'_id': 'Tesco', 'minimum_order_price': '35'}
{'_id': 'Aldi', 'minimum_order_price': '30'}
{'_id': "Sainsbury's", 'minimum_order_price': '35'}
{'_id': 'Lidl', 'minimum_order_price': '30'}
{'_id': 'Asda', 'minimum_order_price': '30'}


In [15]:
#Update Price of Product

myquery = { "$and":[ {"supermarket_name": "Morrisons"}, {"product_name": "Juice"}]}
newvalues = { "$set": { "price_in_pounds": "14" } }

mycol.update_one(myquery, newvalues)

#print documents in the collection after the update:
for x in mycol.find(myquery):
    print(x)

{'_id': ObjectId('65fd836b7b2f6691a83bbfa0'), 'product_id': 'M10020', 'product_name': 'Juice', 'price_in_pounds': '13', 'category_id': '107', 'category_name': 'Drinks', 'supermarket_id': '4', 'supermarket_name': 'Morrisons', 'image_url': 'https://example.com/products/M10020.jpg', 'description': 'Juice from Spain', 'dietary_restrictions': '', 'nutritional_information': "{'calories': 188, 'fat': 17.442598402535168, 'protein': 7.274222877293967, 'carbohydrates': 13.059426153420784}", 'Offers': 'Offer Closing in 3 days', 'minimum_order_price': '40', 'customer_tags': 'affordable', 'best_before_date': '23/03/2024'}


In [32]:
mylist1 = [
   {
  "product_id": "P76543",
  "product_name": "Sparkling Water",
  "price_in_pounds": "5",
  "category_id": "104",
  "category_name": "Drinks",
  "supermarket_id": "2",
  "supermarket_name": "Tesco",
  "image_url": "https://example.com/products/P76543.jpg",
  "description": "Refreshing sparkling water from France",
  "dietary_restrictions": "Vegan",
  "nutritional_information": {
    "calories": 0,
    "fat": 0,
    "protein": 0,
    "carbohydrates": 0
  },
  "Offers": "Buy one get one free",
  "minimum_order_price": "10",
  "customer_tags": "budget-friendly",
  "best_before_date": "15/06/2024"
},
   {
  "product_id": "C20568",
  "product_name": "Organic Green Tea",
  "price_in_pounds": "8",
  "category_id": "109",
  "category_name": "Tea & Coffee",
  "supermarket_id": "3",
  "supermarket_name": "Sainsbury's",
  "image_url": "https://example.com/products/C20568.jpg",
  "description": "Premium organic green tea leaves sourced from Japan",
  "dietary_restrictions": "Organic, Gluten-free",
  "nutritional_information": {
    "calories": 0,
    "fat": 0,
    "protein": 0,
    "carbohydrates": 0
  },
  "Offers": "10% off for loyalty card holders",
  "minimum_order_price": "15",
  "customer_tags": "health-conscious",
  "best_before_date": "30/09/2024"
}]

var = mycol.insert_many(mylist1)

#print list of the _id values of the inserted documents:
print(var.inserted_ids ,"Data inserted")

[ObjectId('65fd86e27b2f6691a83bbfe1'), ObjectId('65fd86e27b2f6691a83bbfe2')] Data inserted


In [33]:
myquery = { "supermarket_name": {"$regex": "^T"} }
myquery2 = {}
# Delete documents matching the query
x = mycol.delete_many(myquery)
print(x.deleted_count, " documents deleted.")

# Define the aggregation pipeline
pipeline = [
    {"$match": myquery2},  # Use an empty dictionary to match all documents
    {"$group": {"_id": "$supermarket_name"}}  # Group by supermarket_name
]

# Execute the aggregation pipeline
distinct_records = mycol.aggregate(pipeline)

# Display distinct supermarket_names
for record in distinct_records:
    print(record['_id'])

1  documents deleted.
M&S
Lidl
Morrisons
Waitrose
Asda
Aldi
Sainsbury's


In [34]:
#Python MongoDB Drop Collection - You can delete a collection by using the drop() method. All deleted indexes
mycol.drop()

#Check if the collection exists by calling the user function defined earlier
check_CollectionExists("GroupB_Supermarket", "ProductDetails", mydb)

The collection  ProductDetails  does not exist in database  GroupB_Supermarket


In [35]:
#Drop database using instance of MongoClient
mongoclient.drop_database("GroupB_Supermarket")

#Check to see if database exists by calling the user function defined earlier 
check_DatabaseExists("GroupB_Supermarket")

The database  GroupB_Supermarket  does not exist.
